In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

train.shape

(42000, 785)

In [3]:
labels = train['label']

In [4]:
labels.dtype

dtype('int64')

In [5]:
X = train.drop(['label'], axis=1)
X.shape

(42000, 784)

图像数据归一化。

In [6]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(X)

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
X_scaler = ss.transform(X)

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


类别变量独热编码。

In [8]:
from keras.utils import to_categorical

y_train = to_categorical(labels) 

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


使用 $3$ 层全连接神经网络训练模型。

In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout

input_dim = X_scaler.shape[1]
nb_classes = y_train.shape[1]

model = Sequential()
model.add(Dense(128, input_dim=input_dim))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# 我们对数据直接使用了交叉验证
model.fit(
    X_scaler,
    y_train,
    nb_epoch=10,
    batch_size=16,
    validation_split=0.1,
    verbose=2)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 37800 samples, validate on 4200 samples
Epoch 1/10
 - 7s - loss: 0.3506 - val_loss: 0.2229
Epoch 2/10
 - 6s - loss: 0.2379 - val_loss: 0.2298
Epoch 3/10
 - 7s - loss: 0.2492 - val_loss: 0.2242
Epoch 4/10
 - 6s - loss: 0.2675 - val_loss: 0.2649
Epoch 5/10
 - 6s - loss: 0.2733 - val_loss: 0.2495
Epoch 6/10
 - 6s - loss: 0.2727 - val_loss: 0.2833
Epoch 7/10
 - 6s - loss: 0.3110 - val_loss: 0.3087
Epoch 8/10
 - 6s - loss: 0.3140 - val_loss: 0.3349
Epoch 9/10
 - 6s - loss: 0.3560 - val_loss: 0.3660
Epoch 10/10
 - 6s - loss: 0.3578 - val_loss: 0.3705


很快就训练完了。

In [10]:
X_test = ss.transform(test)

/Users/liwei/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


In [11]:
preds = model.predict_classes(X_test, verbose=2)

In [12]:
result = pd.DataFrame({"ImageId": list(range(1, preds.shape[0] + 1)), "Label": preds})

In [13]:
result.to_csv('submission.csv', index=False, header=True)